In [ ]:
!apt-get update -qq
!apt-get install -y aria2
!pip install -q msal requests tqdm
!curl https://rclone.org/install.sh | bash

# Movie Upload from URL


In [ ]:
import os
from urllib.parse import urlparse, unquote
import subprocess

DOWNLOAD_URL = input("Enter Download URL: ")

def get_filename_from_aria2(url):
    try:
        result = subprocess.run(
            ['aria2c', '--dry-run', url],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            check=True
        )

        dir = os.getcwd().replace("\\", "/")
        fname = result.stdout.split(dir+"/")[-1].split("\n")[0]
        return fname

    except:
	    print("Error")

filename = get_filename_from_aria2(DOWNLOAD_URL)

print(f"Detected Filename: {filename}")

c = input("Want to change filename? (y/n): ")

if c == "y":
    filename = input("Enter new filename: ")

print("Downloading:", filename)

!aria2c -x 16 -s 16 -k 1M -o "$filename" "$DOWNLOAD_URL"

print("Download complete.")

## Rclone (Faster)

In [ ]:
from google.colab import files
print("Upload rclone.conf file: ", end="")
uploaded = files.upload()

print("\nRclone Remotes: ")
!rclone --config=/content/rclone.conf listremotes

In [ ]:
# ========= EDIT REMOTE NAME ACCORDING TO RCLONE CONFIG ==========
remote_name = "odshrey" # Only the name, no colons
# ========= EDIT REMOTE NAME ACCORDING TO RCLONE CONFIG ==========

cmd = [
    "rclone", "copy", filename, f"{remote_name}:/Movies",
    "--config=/content/rclone.conf",
    "-v",
    "--stats", "5s",
    "--transfers", "4",
    "--checkers", "4",
    "--onedrive-chunk-size", "100M",
    "--tpslimit", "10",
    "--retries", "10"
]

# This runs the command and prints output line-by-line in real time
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
for line in process.stdout:
    print(line, end='')
process.wait()

## Graph API

In [ ]:
import msal
import requests

# =========================
# EDIT THESE VALUES
# =========================

TENANT_ID = ""
CLIENT_ID = ""

# =========================
# DON'T EDIT AFTER HERE
# =========================

AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"
SCOPES = ["Files.ReadWrite.All", "User.Read"]

app = msal.PublicClientApplication(
    CLIENT_ID,
    authority=AUTHORITY
)

flow = app.initiate_device_flow(scopes=SCOPES)

print(flow["message"])  # VERY IMPORTANT: follow this link

result = app.acquire_token_by_device_flow(flow)

if "access_token" not in result:
    raise Exception("Authentication failed")

access_token = result["access_token"]
print("Authentication successful!")


In [ ]:
import os
from tqdm import tqdm

file_size = os.path.getsize(filename)
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

ONEDRIVE_UPLOAD_PATH = f"Movies/{filename}"

# Create upload session
session_url = f"https://graph.microsoft.com/v1.0/me/drive/root:/{ONEDRIVE_UPLOAD_PATH}:/createUploadSession"

response = requests.post(
    session_url,
    headers=headers,
    json={
        "item": {
            "@microsoft.graph.conflictBehavior": "replace"
        }
    }
)

upload_url = response.json()["uploadUrl"]

chunk_size = 10 * 1024 * 1024  # 10 MB
uploaded = 0

with open(filename, "rb") as f:
    with tqdm(total=file_size, unit="B", unit_scale=True) as pbar:
        while uploaded < file_size:
            chunk = f.read(chunk_size)
            start = uploaded
            end = uploaded + len(chunk) - 1

            headers = {
                "Content-Length": str(len(chunk)),
                "Content-Range": f"bytes {start}-{end}/{file_size}"
            }

            r = requests.put(upload_url, headers=headers, data=chunk)
            r.raise_for_status()

            uploaded += len(chunk)
            pbar.update(len(chunk))

print("Upload completed successfully!")


# Series Upload from Zip File

In [ ]:
import os
from urllib.parse import urlparse, unquote
import subprocess

DOWNLOAD_URL = input("Enter zip download URL: ")

def get_filename_from_aria2(url):
    try:
        result = subprocess.run(
            ['aria2c', '--dry-run', url],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            check=True
        )

        dir = os.getcwd().replace("\\", "/")
        fname = result.stdout.split(dir+"/")[-1].split("\n")[0]
        return fname

    except:
	    print("Error")

filename = get_filename_from_aria2(DOWNLOAD_URL)

print(f"Detected Filename: {filename}")

c = input("Want to change filename? (y/n): ")

if c == "y":
    filename = input("Enter new filename: ")

print("Downloading:", filename)

!aria2c -x 16 -s 16 -k 1M -o "$filename" "$DOWNLOAD_URL"

print("Download complete.")

In [ ]:
print(f"Unzipping {filename}")
!unzip "$filename"

In [ ]:
folderpath = input("Copy Extracted Folder Path and Paste Here: ")
sfolder = folderpath.split('/')[-1]

from google.colab import files
print("Upload rclone.conf file: ", end="")
uploaded = files.upload()

print("\nRclone Remotes: ")
!rclone --config=/content/rclone.conf listremotes

In [ ]:
# ========= EDIT REMOTE NAME ACCORDING TO RCLONE CONFIG ==========
remote_name = "odshrey" # Only the name, no colons
# ========= EDIT REMOTE NAME ACCORDING TO RCLONE CONFIG ==========

cmd = [
    "rclone", "copy", folderpath, f"{remote_name}:/Series/{sfolder}/",
    "--config=/content/rclone.conf",
    "-v",
    "--stats", "5s",
    "--transfers", "4",
    "--checkers", "4",
    "--onedrive-chunk-size", "100M",
    "--tpslimit", "10",
    "--retries", "10"
]

# This runs the command and prints output line-by-line in real time
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
for line in process.stdout:
    print(line, end='')
process.wait()